## Prueba de Conocimientos Ciencia de Datos 01

En este notebook puedes utilizar todas las librerias con las que hemos trabajado.

In [1]:
import requests 

from bs4 import BeautifulSoup

from selenium import webdriver

from time import sleep

import helium

import pandas as pd
import numpy as np

KeyboardInterrupt: 

### Ejercicio 1:

_**url** = https://www.20minutos.es/_

Vamos a hacer Web Scraping de las primeras 3 paginas de las categorias de noticias: **Ciencia**, **Deporte**, **Gente**, **Economía**, **Madrid** y **Opinión**.

Y de cada noticia/articulo vamos a obtener:

- **Titulo**
- **Hora**
- **Fecha**
- **Autor**
- **Texto completo**
- **Categoria (columna objetivo)**

**Sustituir con NaN's en el caso de que no se pueda extraer alguno de esos datos anteriores.**

Genera un DataFrame con esta información y guardalo en el archivo **`20minutos_1.csv`**.

In [ ]:
# Driver de Chrome.
chrome_driver = "chromedriver.exe"

In [ ]:
browser = webdriver.Chrome(executable_path = chrome_driver) # Abre el navegador.

browser.get("https://www.20minutos.es/") # Entra en 20minutos.

browser.maximize_window() # Maximiza la ventana del navegador.

In [ ]:
# Para aceptar las cookies.
browser.find_element_by_id("didomi-notice-agree-button").click()
sleep(3)

In [ ]:
# Para acceder a categorías. 
browser.find_element_by_css_selector("#ui-header-land > div.section-menu > nav > ul > li.primary-more.link.current.no-expandable.last").click()
sleep(3)

In [ ]:
# Para entrar en la categoría "Ciencia".
browser.find_element_by_link_text("Ciencia").click()
sleep(3)

In [ ]:
#Extraemos el html y lo pasamos a objeto bs4
soup = BeautifulSoup(browser.page_source, "html.parser")

In [ ]:
def noticiasCategoria(url_original, categoría, número_páginas):
    lista_urls = []
    
    # Para recorrer las páginas.
    for i in range(1, número_páginas + 1):
        
        url = f"{url_original}{categoría}/{i}/"
        print(url)
        
        # EL response con Helium.
        browser = helium.start_chrome(url, headless = True)
        sleep(5)
        
        soup = BeautifulSoup(browser.page_source, "html.parser")
        
        # Para recorrer las noticias.
        for bs in soup.find_all("div", class_ = "media-content"):

            # De esta forma se accede al enlace.
            enlace = bs.find("a")["href"]

            # Para incluir en la lista, solo cuando el enlace incluye el string "noticia", o la categoría se encuentr en 
            # el enlace, además de no tener el string "imagenes".
            if (("noticia" in enlace) or (categoría in enlace)) and ("/imagenes/" not in enlace):
                lista_urls.append(enlace)
    
    return lista_urls

In [ ]:
# Con esto, se extraen las noticias de las 3 primeras páginas, y se guardan en una variable.
# Con el len del final, se comprueba su número.
resultado = noticiasCategoria("https://www.20minutos.es/", "ciencia", 3)
len(resultado)

In [ ]:
%%time

# Con este código, lo mismo de antes, pero para todas las categorías que pide el ejercicio.

categorias = ["ciencia", "deportes", "gente", "economia", "madrid", "opinion"]

dict_enlaces = {}

numero_paginas = 3

for categoria in categorias:
    
    dict_enlaces[categoria] = noticiasCategoria("https://www.20minutos.es/", categoria, numero_paginas)

In [ ]:
# Para ver cuantas noticias son.
len(dict_enlaces[categoria])

In [ ]:
df = pd.DataFrame(dict_enlaces.items(), columns = ["Categoría", "Enlace"])
df

In [ ]:
df = df.explode("Url").reset_index(drop = True)
df

In [ ]:
df.to_csv("urls_ejercicio1_parte1.csv", index = False, sep = ",")

- Escribe el código para sacar la información de las primeras 5 páginas de todas las categorias:
    - **Andalucia**, **Valencia**, **Animaleros**, **Cataluña**, **Nacional**, **Vivienda**, **Internacional**, **Cultura**, **Empleo**, **Ciencia**, **Deporte**, **Gente**, **Economía**, **Madrid** y **Opinión**.

. Guarda esta información en el archivo **`20minutos_2.csv`**.

- **Escribe el código para sacar la información de todas las categorias del ejercicio anterior pero solo del mes de marzo. Guarda esta información en el archivo `20minutos_3.csv`.**

- **Elige el articulo que más te interese de todos los articulos extraidos, muestra en un WordCloud las palabras más repetidas eliminando los stopwords.**

- Define una función que eliga al azar **`n`** artículos al azar del fichero **`20minutos_1.csv`** y que muestre para cada articulo:
    - **Titulo, fecha y el WordCloud.**
    
Prueba la función con **`n = 5`**